In [1]:
from pathlib import Path

from trackastra.model import Trackastra

c:\Users\Cyril\anaconda3\envs\tastra\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.3.dev23+gde041ff 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.6.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [2]:
# model_path = Path(r"C:\Users\Cyril\Desktop\Code\thesis_misc\eval_scripts\BENCHMARK\general\RUNS\2025-07-22_12-00-55_zih_general_v4_sam21_aug")
model_path = Path(
    r"C:\Users\Cyril\Desktop\Code\thesis_misc\eval_scripts\BENCHMARK\bacteria\RUNS\2025-06-24_20-10-57_vanvliet_sam21_aug_norm"
)

In [3]:
DATA_PATH = Path(
    r"C:\Users\Cyril\Desktop\Code\thesis_misc\CTC_DATA\vanvliet\rpsM\151029_E1-1"
)

In [4]:
model = Trackastra.from_folder(model_path)

INFO:trackastra.model.model:Loading model state from C:\Users\Cyril\Desktop\Code\thesis_misc\eval_scripts\BENCHMARK\bacteria\RUNS\2025-06-24_20-10-57_vanvliet_sam21_aug_norm\model.pt
INFO:trackastra.model.model_api:Using device cuda
INFO:trackastra.model.model_api:Default batch size = 4 for model on cuda.


In [5]:
graph, masks_tra = model.track_from_disk(DATA_PATH / "img", DATA_PATH / "TRA")

INFO:trackastra_pretrained_feats.pretrained_features:Using model facebook/sam2.1-hiera-base-plus with mode mean_patches_exact for pretrained feature extraction.
INFO:root:Loaded checkpoint sucessfully
INFO:trackastra.model.model_api:Predicting weights for candidate graph
INFO:trackastra.data.wrfeat:Extracting features from 53 frames.
INFO:trackastra.model.model_api:Building windows                     
Building windows: 100%|██████████| 50/50 [00:00<00:00, 3439.70it/s]
INFO:trackastra.model.model_api:Predicting windows
Computing associations: 100%|██████████| 13/13 [00:01<00:00, 10.68it/s]
INFO:trackastra.model.model_api:Running greedy tracker
INFO:trackastra.tracking.tracking:Build candidate graph with delta_t=1
INFO:trackastra.tracking.tracking:Added 3519 vertices, 3553 edges                         
INFO:trackastra.tracking.tracking:Running greedy tracker
Greedily matched edges:  99%|█████████▉| 3530/3553 [00:00<00:00, 95109.55it/s]


In [6]:
from trackastra.tracking import graph_to_ctc, graph_to_napari_tracks

In [7]:
ctc_tracks, ctc_masks = graph_to_ctc(
    graph,
    masks_tra,
    outdir="tracked_ctc",
)

Saving masks: 100%|██████████| 53/53 [00:00<00:00, 167.96it/s]


In [8]:
napari_tracks, napari_tracks_graph, _ = graph_to_napari_tracks(graph)

100%|██████████| 714/714 [00:00<00:00, 5084436.43it/s]


In [9]:
import napari

v = napari.Viewer()
# v.add_image(imgs)
v.add_labels(masks_tra)
v.add_tracks(data=napari_tracks, graph=napari_tracks_graph)

<Tracks layer 'napari_tracks' at 0x21e8f5b62c0>

In [10]:
import pprint as pp
import napari
import numpy as np
from tqdm import tqdm
from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher
from traccuracy.metrics import CTCMetrics

from divisualisation import Divisualisation
from divisualisation.utils import load_tiff_timeseries, rescale_intensity

In [11]:
gt = load_ctc_data(
    DATA_PATH / "TRA",
    DATA_PATH / "TRA/man_track.txt",
    name="GT",
)
pred = load_ctc_data(
    "./tracked_ctc",
    "./tracked_ctc/man_track.txt",
    name="res",
)

img = load_tiff_timeseries(DATA_PATH / "img")

Loading TIFFs:   0%|          | 0/52 [00:00<?, ?it/s]

Computing node attributes: 100%|██████████| 53/53 [00:00<00:00, 99.91it/s] 
INFO:traccuracy.loaders._ctc:Running CTC format checks
Computing node attributes: 100%|██████████| 53/53 [00:00<00:00, 95.92it/s] 
INFO:traccuracy.loaders._ctc:Running CTC format checks


In [12]:
img = np.stack([
    rescale_intensity(_x, pmin=5, pmax=99.9, clip=False, subsample=1)
    for _x in tqdm(img, desc="Rescale intensity")
])

ctc_results, ctc_matched = run_metrics(
    gt_data=gt,
    pred_data=pred,
    matcher=CTCMatcher(),
    metrics=[CTCMetrics()],
)
pp.pprint(ctc_results)

gt_graph = ctc_matched.gt_graph
pred_graph = ctc_matched.pred_graph

v = napari.current_viewer()
if v is not None:
    v.close()
v = napari.Viewer()
for layer in v.layers:
    v.layers.remove(layer)
v.theme = "dark"

divis = Divisualisation(
    z_scale=1,
    time_scale=12,
    tracks_width=2,
)

v = divis.visualize_gt(
    v,
    x=img,
    masks=pred.segmentation,
    # networkx graph at traccuracy.TrackingGraph.graph
    graph=gt_graph.graph,
)

v = divis.visualize_edge_errors(
    viewer=v,
    gt_graph=gt_graph,
    pred_graph=pred_graph,
    masks_original=gt.segmentation,
    masks_tracked=pred.segmentation,
)

Matching frames: 100%|██████████| 53/53 [00:01<00:00, 49.20it/s] 
INFO:traccuracy.matchers._base:Matched 3519 out of 3519 ground truth nodes.
INFO:traccuracy.matchers._base:Matched 3519 out of 3519 predicted nodes.
Evaluating FN edges: 100%|██████████| 3517/3517 [00:00<00:00, 344811.18it/s]


[{'gt': 'GT',
  'matcher': {'matching type': 'one-to-one', 'name': 'CTCMatcher'},
  'metric': {'e_weights': {'fn': 1.5, 'fp': 1, 'ws': 1},
             'name': 'CTCMetrics',
             'v_weights': {'fn': 10, 'fp': 1, 'ns': 5},
             'valid_match_types': ['one-to-one', 'many-to-one']},
  'pred': 'res',
  'results': {'AOGM': 14.5,
              'DET': 1.0,
              'TRA': 0.9996416700646229,
              'fn_edges': 5,
              'fn_nodes': 0,
              'fp_edges': 5,
              'fp_nodes': 0,
              'ns_nodes': 0,
              'ws_edges': 2},
  'version': '0.2.0'}]


100%|██████████| 714/714 [00:00<00:00, 95562.35it/s]
INFO:divisualisation.divisualisation:Adding gt tracks


INFO:OpenGL.acceleratesupport:No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
c:\Users\Cyril\anaconda3\envs\tastra\lib\site-packages\napari\layers\utils\plane.py:35: RuntimeWarning: invalid value encountered in divide
  return cast(Point3D, tuple(v / np.linalg.norm(v)))
c:\Users\Cyril\anaconda3\envs\tastra\lib\site-packages\napari\layers\utils\plane.py:35: RuntimeWarning: invalid value encountered in divide
  return cast(Point3D, tuple(v / np.linalg.norm(v)))
c:\Users\Cyril\anaconda3\envs\tastra\lib\site-packages\napari\layers\utils\plane.py:35: RuntimeWarning: invalid value encountered in divide
  return cast(Point3D, tuple(v / np.linalg.norm(v)))
c:\Users\Cyril\anaconda3\envs\tastra\lib\site-packages\napari\layers\utils\plane.py:35: RuntimeWarning: invalid value encountered in divide
  return cast(Point3D, tuple(v / np.linalg.norm(v)))
c:\Users\Cyril\anaconda3\envs\tastra\lib\site-packages\napari\layers\utils\plane.py:35: RuntimeWarning: invalid value encounte